# Stellar Tracking by Component

### Header

In [25]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from multiprocessing import Pool

In [26]:
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np

In [27]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [6]:
figure_setup()
settings = Settings()

### Constants

In [7]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

### Methods

In [8]:
def get_ids_by_component(simulation: str) -> pd.DataFrame:
    """
    Return a DataFrame with data of interest.
    """
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    ids = {}
    for component in settings.components:
        is_component = s.region_tag == settings.component_tags[component]
        ids[component] = s.ids[is_component]

    return ids

In [38]:
def get_temporal_evolution_of_stellar_fraction(
    simulation: str) -> pd.DataFrame:

    # Get IDs of stars by component at z=0
    ids = get_ids_by_component(f"{simulation}_s127")

    # Get temporal evolution of the stellar fraction
    columns = [f"{c1}_{c2}" for c1 in settings.components \
        for c2 in settings.components]
    df = pd.DataFrame(np.nan * np.ones((128, 16)))
    df = df.set_axis(columns, axis=1)

    # Add time to dataframe
    temporal_data = pd.read_csv(f"../results/{simulation}/temporal_data.csv")
    df["Time_Gyr"] = temporal_data["Time_Gyr"]

    for i in range(settings.first_snap, 127 + 1):
        this_ids = get_ids_by_component(f"{simulation}_s{i}")
        for j, c1 in enumerate(settings.components):
            n_comp = len(ids[c1])
            for k, c2 in enumerate(settings.components):
                frac = len(set(ids[c1]).intersection(
                    set(this_ids[c2]))) / n_comp
                df.loc[i, f"{c1}_{c2}"] = frac
    
    df.to_csv(f"../results/{simulation}/stellar_tracking{SUFFIX}.csv")
    return df

### Analyse Data

In [39]:
SIMULATIONS = [f"au{i}_or_l4" for i in settings.groups["Included"]]
Pool().map(get_temporal_evolution_of_stellar_fraction, SIMULATIONS)

### Plot of the Temporal Evolution of Stellar Tracking

In [36]:
SIMULATIONS = ["au6_or_l4"]
for SIMULATION in SIMULATIONS:
    df = pd.read_csv(f"../results/{SIMULATION}/stellar_tracking{SUFFIX}.csv")

    fig = plt.figure(figsize=(7.4, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.tick_params(which='both', direction="in")
        ax.set_xlim(0, 14)
        ax.set_ylim(0, 1)
        ax.set_xticks([2, 4, 6, 8, 10, 12])
        ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
        ax.set_xlabel('Time [Gyr]')
        ax.set_ylabel(r"$f_\star$")
        ax.grid(True, ls='-', lw=0.25, c="gainsboro")
        ax.label_outer()

    for i, c1 in enumerate(settings.components):
        axs[i].text(
            x=0.05, y=1.02, size=8.0, ha="left", va="bottom",
            s=r"$\textbf{" \
                + list(settings.component_labels.values())[i] + "}$",
            c=list(settings.component_colors.values())[i],
            transform=axs[i].transAxes)
        for j, c2 in enumerate(settings.components):
            axs[i].plot(
                df["Time_Gyr"], df[f"{c1}_{c2}"], zorder=12,
                c=settings.component_colors[c2], lw=1.5,
                label=settings.component_labels[c2])
        no_comp = 1 - df[f"{c1}_H"] - df[f"{c1}_B"] \
            - df[f"{c1}_WD"] - df[f"{c1}_CD"]
        axs[i].plot(
            df["Time_Gyr"], no_comp, zorder=12, c="black", lw=1.5)

    axs[2].legend(loc="center left", framealpha=0, fontsize=6,
                  bbox_to_anchor=(0.01, 0.5))

    axs[0].text(
        x=0.95, y=1.02, size=8.0,
        s=r"$\texttt{" + f"Au{parse(SIMULATION)[0]}" + "}$",
        ha="right", va="bottom", transform=axs[0].transAxes)

    fig.savefig(
        f"../images/stellar_tracking/{SIMULATION}{SUFFIX}.pdf")
    plt.close(fig)